## Importing

In [1]:
from dataset import PressureUlcers
import torchvision.transforms as ttran
import torch
from torch.utils.data import DataLoader
from dataset import get_class_map

## Data Loading

### Loading Dataset

In [2]:
IMG_SIZE = (448, 448) # (448, 448)

JSON_MAPPING_PATH = '../data/unzipped/notes.json'
transform = ttran.Compose([ttran.Resize(IMG_SIZE), ttran.ToTensor(), ttran.Normalize((0, 0, 0), (1, 1, 1))]) # ttran.ColorJitter()


class_map = get_class_map(JSON_MAPPING_PATH)

ds = PressureUlcers('../data/images', '../data/unzipped/labels', transform, grid_size=7, num_bounding_boxes=2, img_size=IMG_SIZE, num_classes=len(class_map))

folder: Stage_III
files: ['Stage_III_045.png', 'Stage_III_034.png', 'Stage_III_022.png', 'Stage_III_006.png', 'Stage_III_018.png', 'Stage_III_031.png', 'Stage_III_008.png', 'Stage_III_005.png', 'Stage_III_033.png', 'Stage_III_021.png', 'Stage_III_028.png', 'Stage_III_042.png', 'Stage_III_014.png', 'Stage_III_025.png', 'Stage_III_040.png', 'Stage_III_015.png', 'Stage_III_016.png', 'Stage_III_037.png', 'Stage_III_003.png', 'Stage_III_027.png', 'Stage_III_030.png', 'Stage_III_010.png', 'Stage_III_013.png', 'Stage_III_039.png', 'Stage_III_038.png', 'Stage_III_035.png', 'Stage_III_017.png', 'Stage_III_001.png', 'Stage_III_046.png', 'Stage_III_026.png', 'Stage_III_029.png', 'Stage_III_024.png', 'Stage_III_032.png', 'Stage_III_011.png', 'Stage_III_004.png', 'Stage_III_043.png', 'Stage_III_041.png', 'Stage_III_009.png', 'Stage_III_007.png', 'Stage_III_019.png', 'Stage_III_002.png', 'Stage_III_044.png', 'Stage_III_012.png', 'Stage_III_023.png', 'Stage_III_020.png', 'Stage_III_036.png']


folder

## Setting Up Device

In [3]:
from torch import device

torch_device = device('cuda' if torch.cuda.is_available() else 'cpu')

## Creating Model

In [4]:
from model import YOLOModel
from simple_yolo_model import SimpleYOLOModel
model = SimpleYOLOModel(num_classes=len(class_map), img_size=IMG_SIZE[0])

## Testing For Single Image

In [5]:
tensor_img, labels = ds[120]
# Putting it in a batch of its own
tensor_img = tensor_img.reshape(1, tensor_img.shape[0], tensor_img.shape[1], tensor_img.shape[2] )

iter_dl = iter(DataLoader(dataset=tensor_img, batch_size=1, shuffle=True)) 

# About using dataloader and having it convert [H, W, C] to [C, H, W] (IAmIronMan's comment)
# https://discuss.pytorch.org/t/runtimeerror-given-groups-1-weight-of-size-64-3-7-7-expected-input-3-1-224-224-to-have-3-channels-but-got-1-channels-instead/30153/8

model = model.eval()

result = model(next(iter_dl))

result.shape
# 7x7 grid. Each value is composed of 5 bounding boxes (each with x, y, w, h) 
# and class probabilities for each of the 20 classes
# Output size = S x S x (B * 5 + C)

torch.Size([1, 7, 7, 46])

## Setting Hyperparameters

In [6]:
# https://www.ultralytics.com/glossary/intersection-over-union-iou
# A threshold determines if the prediction is acceptable

def IOU(boxes, ground_truth_boxes):
    """
        Intersection over Union implementation.
        It is used to evaluate how well the predicted bounding box is in comparison with the ground truth.

        $ IOU = (Intersection)/(Union)

        ## Args

        - *boxes*: array-like of boxes. Each box is a list of c, x, y, w, h.
        - *ground_truth_boxes*: array-like of boxes. Each box is a list of c, x, y, w, h
    """

    # Between two boxes
    pass


In [7]:
import torch
from torchvision.ops import box_iou
from yolo_loss import yolo_loss

LEARNING_RATE = 0.001
SCHEDULER_STEP_SIZE = 10
SCHEDULER_GAMMA = 0.1
BATCH_SIZE = 25 # about 25 is the limit for SimpleYoloModel with our current gpu
NUM_EPOCHS = 15

optimizer = torch.optim.Adadelta(params=model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, SCHEDULER_STEP_SIZE, SCHEDULER_GAMMA)
loss = yolo_loss

## Training

### Splitting into Batches

In [8]:
from torch.utils.data import random_split

# from sklearn.model_selection import train_test_split
# train_indices, test_indices = train_test_split(range(len(ds)), test_size=0.2, random_state=42)

train_ds, test_ds = random_split(ds, [0.8, 0.2], torch.Generator().manual_seed(42)) # https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

### Training Loop

In [9]:
from tqdm import tqdm
import math
from torch.nn.utils import clip_grad_norm_, clip_grad_value_

NUM_BATCHES = math.ceil(len(train_ds) / BATCH_SIZE)

model.train()

model.to(torch_device)

# For each epoch
for epoch in tqdm(range(NUM_EPOCHS)):
    
    # Load dataset
    iter_dl = iter(DataLoader(dataset=train_ds, batch_size=BATCH_SIZE, shuffle=True))

    final_loss = 0

    # For each batch
    for batch_i in range(NUM_BATCHES):

        # Load data
        img_data, labels = next(iter_dl)

        # Put it on GPU
        img_data = img_data.to(torch_device)
        labels = labels.to(torch_device)

        # Make predictions
        predictions = model(img_data)

        # Calculate Loss
        loss_res = yolo_loss(labels, predictions, num_classes=model.num_classes)

        # Optimize
        optimizer.zero_grad()
        loss_res.backward()
        # clip gradient to avoid exploding gradients
        #clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()
        
        final_loss += loss_res.item() # Use of .item (pandas007): https://stackoverflow.com/questions/59129812/how-to-avoid-cuda-out-of-memory-in-pytorch
    print(f'Epoch {epoch} | Loss: {final_loss}')
    
# https://discuss.pytorch.org/t/why-my-model-returns-nan/24329/6
# https://medium.com/biased-algorithms/guide-to-gradient-clipping-in-pytorch-f1db24ea08a2
# RuntimeError: The size of tensor a (23) must match the size of tensor b (0) at non-singleton dimension 0

  7%|▋         | 1/15 [00:06<01:34,  6.75s/it]

Epoch 0 | Loss: 2394.027702331543


 13%|█▎        | 2/15 [00:13<01:24,  6.51s/it]

Epoch 1 | Loss: 1599.3828506469727


 20%|██        | 3/15 [00:19<01:16,  6.41s/it]

...

Epoch 12 | Loss: 1516.98237991333


 93%|█████████▎| 14/15 [01:28<00:06,  6.28s/it]

Epoch 13 | Loss: 1591.1847686767578


100%|██████████| 15/15 [01:34<00:00,  6.32s/it]

Epoch 14 | Loss: 1536.5187301635742
